In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from dispkernel import dispKernel

plt.rcParams['figure.figsize'] = (12, 5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
traindata = torch.Tensor(np.loadtxt('traindata.csv', delimiter=',')).to(device)
trainlabel = torch.Tensor(np.loadtxt('trainlabel.csv', delimiter=',')).to(device)
validdata = torch.Tensor(np.loadtxt('validdata.csv', delimiter=',')).to(device)
validlabel = torch.Tensor(np.loadtxt('validlabel.csv', delimiter=',')).to(device)

data = (traindata, trainlabel, validdata, validlabel)

In [ ]:
def get_accuracy(predictions, labels):
    return ((predictions >= 0.5) == labels).sum().item() / labels.size(0)

def train(traindata, trainlabel, validdata, validlabel, activation, lr, epochs, debug=False):
    net = nn.Sequential(nn.Linear(9, 1), activation()).to(device) # Our neuron is a nn.Linear followed by an activation function

    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    optimizer = torch.optim.SGD(net.parameters(), lr)
    classifier = nn.MSELoss(reduction='sum') # default reduction is mean, but in class we only sum

    for i in range(1, epochs + 1):
        net.train()
        optimizer.zero_grad()
        pred = net(traindata).squeeze(1)
        loss = classifier(pred, trainlabel)
        loss.backward()
        optimizer.step()

        train_loss.append(loss)
        train_acc.append(get_accuracy(pred, trainlabel))

        net.eval()
        pred = net(validdata).squeeze(1)
        loss = classifier(pred, validlabel)
        
        val_loss.append(loss)
        val_acc.append(get_accuracy(pred, validlabel))

        if debug:
            print('epoch {}, train loss {:2f} acc {:2f} validation loss {:2f} acc {:2f}'.format(
                i,
                train_loss[-1],
                train_acc[-1],
                val_loss[-1],
                val_acc[-1]
            ))
        
    return net, train_loss, train_acc, val_loss, val_acc

def plot_history(hyperparameters, train_loss, train_acc, val_loss, val_acc):
    plt.suptitle('Single Neuron Classifier, {} activation function, {} epochs, learning rate = {}'.format(
        hyperparameters['activation'].__name__,
        hyperparameters['epochs'],
        hyperparameters['lr']
    ))
    plt.subplot(1, 2, 1)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.plot(train_loss, label='training')
    plt.plot(val_loss, label='validation')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.plot(train_acc, label='training')
    plt.plot(val_acc, label='validation')
    plt.legend()
    plt.show()

In [ ]:
# LR too low
hyperparameters = {
    'activation': nn.Identity,
    'lr': 0.0000001,
    'epochs': 1000,
    'seed': 0
}

In [ ]:
# LR too high
hyperparameters = {
    'activation': nn.Identity,
    'lr': 0.002,
    'epochs': 100,
    'seed': 0
}

In [ ]:
# LR good
hyperparameters = {
    'activation': nn.Identity,
    'lr': 0.0010,
    'epochs': 100,
    'seed': 0
}

This cell runs the training loop with the defined hyperparameters, and plots the training curves.

In [ ]:
torch.manual_seed(hyperparameters['seed'])
neuron, train_loss, train_acc, val_loss, val_acc = train(
    traindata, trainlabel, validdata, validlabel,
    hyperparameters['activation'], hyperparameters['lr'], hyperparameters['epochs'])
plot_history(hyperparameters, train_loss, train_acc, val_loss, val_acc)

In [ ]:
dispKernel(neuron.state_dict()['0.weight'].cpu().numpy()[0], 3, 3)

In [ ]:
net.state_dict()